# SSM Challenge
There is an SSM parameter in our Explore-US environment called: /customer/model-retention/ai/db-connections/data-sci-retention/database-read

The challenge is to create a notebook in your sandbox that does the following:
1. Read the value of the SSM parameter
2. Use the values in the SSM parameter to form a connection string to the MSSQL database listed in the parameter
3. Read the contents of the ds.customerScores table into a dataframe

In [19]:
import boto3 
import json
import pandas as pd
import psycopg2
import pyodbc

In [20]:
! aws sso login --profile Explore-US-DataScienceAdmin

Attempting to automatically open the SSO authorization page in your default browser.
If the browser does not open or you wish to use a different device to authorize this request, open the following URL:

https://device.sso.us-east-1.amazonaws.com/

Then enter the code:

MFBL-HJTS
Successfully logged into Start URL: https://stellaralgo.awsapps.com/start#/


### Get connection details using SSM and create a SQL connection using those details

In [21]:
ssm_param = "/customer/model-retention/ai/db-connections/data-sci-retention/database-read"

session = boto3.setup_default_session(profile_name='Explore-US-DataScienceAdmin')
        
ssm_client = boto3.client("ssm", "us-east-1")

response = ssm_client.get_parameter(
    Name=ssm_param,
    WithDecryption=True,
)["Parameter"]["Value"]

sql_connection = json.loads(response)

In [22]:
#Hard coded production server as the sql_connection gives an outdated ip address
CNXN = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+'34.206.73.189'+
';DATABASE='+sql_connection["database"]+
';UID='+sql_connection["username"]+
';PWD='+ sql_connection["password"])

### Use the MsSql connection to create a data frame of the CustomerScores table

In [23]:
cursor = CNXN.cursor()
query = f"""SELECT TOP 100 * FROM [ds].[customerScores]"""
CNXN.commit()
cursor.close()

df_customerScores = pd.read_sql(query, CNXN)

display(df_customerScores)

,dimcustomermasterid,buyer_score,tenuredays,attendancePercentage,mostRecentAttendance,year,lkupclientid,productGrouping,seasonYear,insertDate
0,349335093,0.9115,2190,0.555556,2022-01-17,2021,33,Full Season,2021,01-22-2022 09:33:41
1,349335290,0.3554,1825,1.037037,2022-01-17,2021,33,Full Season,2021,01-22-2022 09:33:41
2,349335440,0.3491,1825,1.142857,2022-01-17,2021,33,Partial,2021,01-22-2022 09:33:41
3,349335508,0.359,2190,1.037037,2022-01-17,2021,33,Full Season,2021,01-22-2022 09:33:41
4,349344774,0.8677,1460,0.814815,2022-01-17,2021,33,Full Season,2021,01-22-2022 09:33:41
...,...,...,...,...,...,...,...,...,...,...
95,352986413,0.414,1095,1.000000,2022-01-17,2021,33,Full Season,2021,01-22-2022 09:33:41
96,352986684,0.6225,1460,1.000000,2022-01-06,2021,33,Partial,2021,01-22-2022 09:33:41
97,352987192,0.5432,365,0.851852,2022-01-17,2021,33,Full Season,2021,01-22-2022 09:33:41
98,352987363,0.4904,2190,0.666667,2021-12-12,2021,33,Partial,2021,01-22-2022 09:33:41
